In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

# Import Library

In [ ]:
import pandas as pd
from pandas import DataFrame as df

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim
from gensim.models import Word2Vec #@
from gensim.utils import simple_preprocess #@
from gensim.models.keyedvectors import KeyedVectors #@

#Keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model #모델 저장

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Utility
import re
import numpy as np
import time #수행시간 측정
from google.colab import files #colab에 모델 save,load

# emoji패키지 설치
# 영구 설치 가능, https://sikaleo.tistory.com/m/78?category=932203
!pip install emoji
import emoji

In [ ]:
# =============== 셋팅 =============== #

# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
TRAIN_SIZE = 0.8

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

#전처리
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 8
BATCH_SIZE = 1024

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# Load Data

In [ ]:
my_path = '/gdrive/My Drive/colab/'

In [ ]:
#학습데이터 로드
dataset=pd.read_csv(my_path+'train.csv',encoding = DATASET_ENCODING, names=DATASET_COLUMNS)
print(dataset.shape) #1600000,6

In [ ]:
dataset.head() # negative:0, positive:4

디코더 생성

In [ ]:
decode_map = {0: NEGATIVE, 2: NEUTRAL, 4: POSITIVE} #숫자 => 분류 문장
def decode_sentiment(label):
    return decode_map[int(label)]

In [ ]:
%%time
dataset.target = dataset.target.apply(lambda x: decode_sentiment(x))

# Pre-Processing 

클리닝 텍스트

In [ ]:
#학습 데이터 텍스트 전처리
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [ ]:
#학습 데이터 전처리 진행
dataset.text = dataset.text.apply(lambda x: preprocess(x)) 

학습 데이터 나누기 

In [ ]:
train, test = train_test_split(dataset, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(train))
print("TEST size:", len(test))

In [ ]:
documents = [_text.split() for _text in train.text] #list, 1280000*50

In [ ]:
vocab_size = 400000
tk = Tokenizer(num_words=vocab_size)
tk.fit_on_texts(train.text) 
x_train = tk.texts_to_sequences(train.text)
x_test = tk.texts_to_sequences(test.text)

In [ ]:
labels = train.target.unique().tolist() #POSITIVE NEUTRAL NEGATIVE
labels.append(NEUTRAL)
print(labels)

encoder = LabelEncoder() #문장 -> 숫자 자동으로
encoder.fit(train.target.tolist())

y_train = encoder.transform(train.target.tolist())
y_test = encoder.transform(test.target.tolist())

y_train = y_train.reshape(-1,1) #1열로 자동으로 만들어줍니다.
y_test = y_test.reshape(-1,1)

# Build Model

In [ ]:
max_len=max(len(l) for l in x_train) #한 문장에서 최대 단어 개수를 반환 #max_len=50

In [ ]:
X_train = np.array(pad_sequences(x_train, maxlen=max_len, padding='post')) #max_len만큼 padding 값 설정 
print(X_train.shape, y_train.shape)

text-CNN 모델 로드

In [ ]:
model = load_model(my_path+'text-CNN.h5')

In [ ]:
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

In [ ]:
def predict(ex_text, include_neutral=True):
    start_at = time.time()
    x_encoded = tk.texts_to_sequences([ex_text])
    res_test=np.array(pad_sequences(x_encoded, maxlen=max_len, padding='post'))
    # Predict
    score = model.predict([res_test])
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)
    
    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}

In [ ]:
predict("That's so sad")

# 트위터 데이터로 예측해보기

In [ ]:
# 트윗 텍스트 전처리
def preproc_tweets(request_id):
    #사용자에게 분석 요청 받은 유명인 아이디
    id = request_id

    file_name = 'tweets_'
    fileformat = '.txt'
    filename = file_name + id + fileformat

    data_path = '/gdrive/My Drive/colab/data/'

    # 분석 요청된 유명인 트윗 파일 open
    with open(data_path + filename, 'r', encoding = "utf-8") as f:
        tweets = pd.read_csv(f, sep = "\n", names = ['data'])
    f.close()

    #소문자 변환
    tweets['preprocess'] = tweets['data'].str.lower()

    #@,#제거
    tweets['preprocess'] = tweets['preprocess'].replace(to_replace = "(@|#)", value = " ", regex = True)

    #Emoji 변환
    tweets_raw = tweets['preprocess']

    for i in range(len(tweets_raw)):
        tweets_raw[i] = emoji.demojize(tweets_raw[i], use_aliases = True)

    tweets['preprocess'] = tweets_raw

    return tweets

In [ ]:
# DB에서 분석할 트위터 텍스트 파일 가져오기
# 입력 값 ID 필요
request_id = "@AnneMarie"

tweet_data = preproc_tweets(request_id)

tweet_data


In [ ]:
df_res = pd.DataFrame({'text':[],
                   'label':[],
                   'score':[],
                  'elapsed_time':[]}) #결과 dataframe 생성

In [ ]:
for col,item in tweet_data.iterrows():
  res=predict(item[1])
  df_res.loc[col]=[item[0], res['label'], res['score'],res['elapsed_time'] ]

df_res